<a href="https://colab.research.google.com/github/marianobonelli/NDVI_Downloader/blob/main/Visualizaci%C3%B3n_y_descarga_im%C3%A1genes_NDVI_de_Harmonized_Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab desarrollado por [Mariano Bonelli](https://www.linkedin.com/in/mariano-francisco-bonelli/) para la selección y descarga de imágenes NDVI para un lote específico de la colección [Harmonized Sentinel 2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) de GEE (la colección tiene datos disponibles desde 2017-03-28)

Este colab necesita previamente tener instalada la extensión [Open Earth Engine](https://chrome.google.com/webstore/detail/open-earth-engine-extensi/dhkobehdekjgdahfldleahkekjffibhg) en chrome y permisos para acceder a google earth engine.

En importación de archivos cargar la capa de lotes descargado de 360 o una capa de polígonos genérica que en la primera columna tenga los nombres de los lotes. soporta archivos .shp, .zip o .geojson

Si el punto 1 da error ir a Entorno de ejecución > Desconectarse y eliminar entorno de ejecución y volver a probar

La imagen NDVI descargada queda en la carpetita en el panel de la izquierda de donde la pueden descargar a su computadora

[Video](https://youtu.be/gBYNJrzBDbw) de ejemplo sobre como usarlo


---



In [ ]:
# @title Instalación e importación de librerías
# Inicio automático en engine desde colab
# Requiere instalar e importar oeel (con la extensión de chrome instalada)
# Luego ya se puede importar engine y utilizarlo

!pip install oeel
!pip install geemap
import geemap
from oeel import oeel
oeel.colab.AuthAndInitilize()
import ee
import chardet



---



In [ ]:
#@title Importación de archivos
from google.colab import files
from ipywidgets import Dropdown
uploaded = files.upload()



---



In [ ]:
# @title Selector de lote y rango de fechas para buscar las imágenes disponibles

import glob
import geopandas as gpd

# Definir las extensiones de archivo de interés
file_extensions = ('*.geojson', '*.shp', '*.zip')
files = []

# Buscar archivos con las extensiones especificadas
for extension in file_extensions:
    files.extend(glob.glob(f'/content/{extension}'))

# Comprobar si se encontraron archivos
if len(files) > 0:
    # Seleccionar automáticamente el primer archivo de la lista
    selected_file = files[0]

else:
    print("No se encontraron archivos con las extensiones especificadas.")

# Funcion para leer archivos como geodataframe
# Primero prueba leerlo normalmente, si no puede lo hace con la codificación ISO... y sino usa chardet
def read_gdf(file_path):
    try:
        gdf = gpd.read_file(file_path)
    except UnicodeDecodeError:
        gdf = gpd.read_file(file_path, encoding='ISO-8859-1')
    except Exception as e:
        with open(file_path, 'rb') as f:
            result = chardet.detect(f.read())  # chardet es una librería para detectar la codificación del archivo
            encoding = result['encoding']
        gdf = gpd.read_file(file_path, encoding=encoding)
    return gdf

# Función para reproyectar un geodataframe a partir del utm estimado:
def transform_crs_to_estimated(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    # Estimar el CRS
    estimated_epsg = gdf.estimate_utm_crs()

    # Transformar el CRS del GeoDataFrame al estimado
    transformed_gdf = gdf.to_crs(estimated_epsg)

    return transformed_gdf

# Leemos el archivo del lote (generamos el gdf)

lote_gdf = read_gdf(selected_file)

columna_1_lote_gdf = lote_gdf.iloc[:, 0].tolist()

# Creamos el widget de la lista desplegable
lote = Dropdown(options=columna_1_lote_gdf)
display('Lote:', lote)

################################################################################
import ipywidgets as widgets
from IPython.display import display
import datetime

# Crear el widget de selector de fecha
fecha_1 = widgets.DatePicker(
    disabled=False,
)

# Crear el widget de selector de fecha
fecha_2 = widgets.DatePicker(
    disabled=False,
)

# Mostrar el widget
display('Fecha de inicio:', fecha_1)

# Mostrar el widget
display('Fecha final:', fecha_2)



---



In [ ]:
# @title Selector de imagen a visualizar
fecha_de_inicio = fecha_1.value
fecha_de_finalizacion = fecha_2.value
# ruta_al_archivo_shp_zip = "/content/lote.zip" # @param {type:"string"}

################################################################################
# Filtro de nubes
################################################################################

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img.addBands(is_cld_shdw)

################################################################################

import geopandas as gpd

# # Cargar el archivo del polígono
# gdf = gpd.read_file(ruta_al_archivo_shp_zip)
# Obtener el valor seleccionado del widget
lote_seleccionado = lote.value

# Filtrar el GeoDataFrame por el valor seleccionado
filtro = lote_gdf.iloc[:, 0] == lote_seleccionado
lote_gdf_filtrado = lote_gdf[filtro]


# Extraer la geometría en formato GeoJSON
geom = lote_gdf_filtrado.geometry.iloc[0].__geo_interface__

# Convierte el GeoJSON a una geometría de Earth Engine
ee_geom = ee.Geometry(geom)

AOI = ee.Geometry(geom)
START_DATE = str(fecha_de_inicio)
END_DATE = str(fecha_de_finalizacion)
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

s2_sr_cld_col = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask))

import pandas as pd

def add_ndvi(image):
    image = image.clip(ee_geom)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Apply the add_ndvi function to each image in the collection.
ndvi_collection = s2_sr_cld_col.map(add_ndvi)

################################################################################

import pandas as pd

def compute_mean(image):
    mean_ndvi = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee_geom,
        scale=10,
        maxPixels=1e9,
        bestEffort=True
    ).get('NDVI')
    return ee.Feature(None, {'date': image.date().format(), 'mean_ndvi': mean_ndvi})

# Apply the function to the collection.
mean_features = ndvi_collection.map(compute_mean)

# Convert the FeatureCollection to a list and get the information.
info = mean_features.getInfo()['features']

dates = [feature['properties']['date'] for feature in info if 'mean_ndvi' in feature['properties']]
mean_ndvi_values = [feature['properties']['mean_ndvi'] for feature in info if 'mean_ndvi' in feature['properties']]

# Create a DataFrame with the data.
df = pd.DataFrame({
    'Date': pd.to_datetime(dates),
    'Mean_NDVI': mean_ndvi_values
})

# Redondea los valores de NDVI a 3 decimales
df['Mean_NDVI'] = df['Mean_NDVI'].apply(lambda x: round(x, 3))

# Muestra solo la fecha (sin la hora) en la columna de fecha
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')


################################################################################

# Crear una lista de opciones para el widget desplegable
options = [f"{row['Date']} - NDVI: {row['Mean_NDVI']}" for index, row in df.iterrows()]

# Crear el widget de selección de fecha
date_selector = widgets.Dropdown(
    options=options,
    disabled=False,
)

# Mostrar el widget
display('Imagen a visualizar:', date_selector)



---



In [ ]:
# @title Visualización y descarga NDVI
# Función para extraer la fecha del valor seleccionado
def get_selected_date():
    return date_selector.value.split(' - ')[0]

# Ahora, cuando quieras obtener la fecha seleccionada, simplemente llama a la función get_selected_date
selected_date = get_selected_date()

# Filtrar la colección por la fecha seleccionada
date_filter = ee.Filter.date(ee.Date(selected_date), ee.Date(selected_date).advance(1, 'day'))
filtered_collection = ndvi_collection.filter(date_filter)

# Obtén la primera imagen de la colección filtrada (si hay más de una en esa fecha, esto tomará la primera)
image = filtered_collection.first()

# Comprueba si la banda NDVI ya está presente, si no, calcula el NDVI
if 'NDVI' not in image.bandNames().getInfo():
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    image = image.addBands(ndvi)

# Obtén las coordenadas centrales del lote seleccionado
center_coords = ee_geom.centroid().coordinates().getInfo()[::-1]  # [lat, lon]

# Crea el mapa
Map = geemap.Map(center=center_coords, zoom=16)

# Parámetros de visualización para NDVI
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': [
        'FFFFFF',
        'CE7E45',
        'DF923D',
        'F1B555',
        'FCD163',
        '99B718',
        '74A901',
        '66A000',
        '529400',
        '3E8601',
        '207401',
        '056201',
        '004C00',
        '023B01',
        '012E01',
        '011D01',
        '011301',
    ]
}

# Parámetros de visualización para True Color y False Color
true_color_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}

false_color_vis = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 5000
}

# Añade las capas al mapa
Map.addLayer(image, true_color_vis, 'True Color')
Map.addLayer(image, false_color_vis, 'False Color')
Map.addLayer(image.select('NDVI'), ndvi_vis, 'NDVI')

# Colorbar NDVI
colors = ndvi_vis['palette']
vmin = ndvi_vis['min']
vmax = ndvi_vis['max']
Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="NDVI")

# Añadir el control de capas
Map.addLayerControl()

# Define una función para descargar la imagen NDVI
def download_ndvi():
    ndvi_img = image.select('NDVI')
    file_path = 'NDVI ' + str(selected_date) + '.tif'
    geemap.ee_export_image(ndvi_img, filename=file_path, scale=10, region=ee_geom, file_per_band=False)

# Crea un botón de descarga
download_button = widgets.Button(description='Descargar NDVI')
download_button.on_click(lambda x: download_ndvi())

# Muestra el botón de descarga
display(download_button)

# Muestra el mapa
Map



---



In [ ]:
# @title Eliminación de archivos para volver a empezar
%%capture
!find /content/ -name "*.zip" -exec rm -r {} \; # elimina todos los archivos .zip del directorio /content/
!find /content/ -name "*.tif" -exec rm -r {} \; # elimina todos los archivos .txt del directorio /content/



---

